In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('./allnfllines.csv', header = 1)

In [3]:
df.head(2)

,Road v Home,Date,Road Open Spread,Road Close Spread,Open Result,Home Open Spread,Home Close Spread,Close Result
0,CAR 20 @ DEN 21,9/8/16,1,-3,PUSH,-1,3,DEN
1,BUF 7 @ BAL 13,9/11/16,2.5,3,BAL,-2.5,-3,BAL


In [4]:
df = df[['Road v Home', 'Date', 'Home Open Spread', 'Home Close Spread']]
df.head(2)

,Road v Home,Date,Home Open Spread,Home Close Spread
0,CAR 20 @ DEN 21,9/8/16,-1,3
1,BUF 7 @ BAL 13,9/11/16,-2.5,-3


In [5]:
df.dtypes

Road v Home          object
Date                 object
Home Open Spread     object
Home Close Spread    object
dtype: object

In [11]:
df = df[df['Road v Home'] != 'Showing 1 to 16 of 16 entries']
df = df[df['Road v Home'] != 'Road v Home']

In [13]:
df[df['Road v Home'].isnull()].head(2) ## there are a bunch, only print first two

,Road v Home,Date,Home Open Spread,Home Close Spread
187,NaN,NaN,NaN,NaN
376,NaN,NaN,NaN,NaN


In [14]:
df = df.dropna()
df.reset_index(drop = True, inplace = True)

In [16]:
home = []
road = []
home_score = []
road_score = []

for i in range(len(df)):
    matchup = df['Road v Home'][i]
    x = matchup.split()
    road.append(x[0])
    road_score.append(x[1])
    home.append(x[3])
    home_score.append(x[4])
    
df['home'] = home
df['road'] = road
df['home_score'] = home_score
df['road_score'] = road_score

In [17]:
df.head(2)

,Road v Home,Date,Home Open Spread,Home Close Spread,home,road,home_score,road_score
0,CAR 20 @ DEN 21,9/8/16,-1,3,DEN,CAR,21,20
1,BUF 7 @ BAL 13,9/11/16,-2.5,-3,BAL,BUF,13,7


In [18]:
try:
    del df['Road v Home']
except:
    pass

In [19]:
df['home'][df['home'] == 'LA'] = 'STL'

In [20]:
df['road'][df['road'] == 'LA'] = 'STL'

In [21]:
len(df['home'].value_counts())

32

In [22]:
df['Home Open Spread'] = df['Home Open Spread'].apply(lambda x: float(x))
df['Home Close Spread'] = df['Home Close Spread'].apply(lambda x: float(x))
df['home_score'] = df['home_score'].apply(lambda x: float(x))
df['road_score'] = df['road_score'].apply(lambda x: float(x))

In [29]:
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
df['DoW'] = df['Date'].apply(lambda x: str(x.dayofweek))

In [30]:
df.head(2)

,Date,Home Open Spread,Home Close Spread,home,road,home_score,road_score,DoW
0,2016-09-08,-1.0,3.0,DEN,CAR,21.0,20.0,3
1,2016-09-11,-2.5,-3.0,BAL,BUF,13.0,7.0,6


In [31]:
big_df = pd.concat([df.drop(['home'], axis = 1), \
                             pd.get_dummies(df['home'], prefix = 'home')], axis = 1)
big_df = pd.concat([big_df.drop(['road'], axis = 1), \
                             pd.get_dummies(big_df['road'], prefix = 'road')], axis = 1)

In [32]:
big_df.head(2)

,Date,Home Open Spread,Home Close Spread,home_score,road_score,DoW,home_ARI,home_ATL,home_BAL,home_BUF,...,road_OAK,road_PHI,road_PIT,road_SD,road_SEA,road_SF,road_STL,road_TB,road_TEN,road_WAS
0,2016-09-08,-1.0,3.0,21.0,20.0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-09-11,-2.5,-3.0,13.0,7.0,6,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
## drop a dummy!

try:
    del big_df['home_ARI']
    del big_df['road_ARI']
except:
    pass

In [36]:
big_df['DoW'].value_counts()

6    2203
0     170
3     133
5      51
1       1
4       1
2       1
Name: DoW, dtype: int64

In [39]:
big_df = big_df[big_df['DoW'] != 1]
big_df = big_df[big_df['DoW'] != 4]
big_df = big_df[big_df['DoW'] != 2]
big_df.reset_index(drop = True, inplace = True)

In [40]:
big_df['DoW'] = big_df['DoW'].apply(lambda x: x.replace('6', 'Sunday'))
big_df['DoW'] = big_df['DoW'].apply(lambda x: x.replace('0', 'Monday'))
big_df['DoW'] = big_df['DoW'].apply(lambda x: x.replace('3', 'Thursday'))
big_df['DoW'] = big_df['DoW'].apply(lambda x: x.replace('5', 'Saturday'))

In [41]:
big_df = pd.concat([big_df.drop(['DoW'], axis = 1), \
                             pd.get_dummies(big_df['DoW'])], axis = 1)
try:
    del big_df['Sunday']
except:
    pass

In [42]:
big_df.head(3) ## get rid of 1, 2, 4 columns

,Date,Home Open Spread,Home Close Spread,home_score,road_score,home_ATL,home_BAL,home_BUF,home_CAR,home_CHI,...,road_STL,road_TB,road_TEN,road_WAS,1,2,4,Monday,Saturday,Thursday
0,2016-09-08,-1.0,3.0,21.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2016-09-11,-2.5,-3.0,13.0,7.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-09-11,4.0,3.5,23.0,27.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
